In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as pads
import os
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
data_path = "/Users/pranavrajaram/SMT-Data-Challenge-2025"

In [3]:
def readDataSubset(table_type, data_path):
    """
    Loads a specified SMT data subset as a PyArrow dataset.
    """
    valid_tables = ['ball_pos', 'game_events', 'game_info', 'player_pos', 'rosters']
    if table_type not in valid_tables:
        print("Invalid data subset name. Please try again with a valid data subset.")
        return None

    if table_type == 'rosters':
        return pads.dataset(source=os.path.join(data_path, 'rosters.csv'), format='csv')
    else:
        
        return pads.dataset(
            source=os.path.join(data_path, table_type),
            format='csv'
        )

In [4]:
game_info_ds = readDataSubset('game_info', data_path)
game_events_ds = readDataSubset('game_events', data_path)
ball_pos_ds = readDataSubset('ball_pos', data_path)
player_pos_ds = readDataSubset('player_pos', data_path)
rosters_ds = readDataSubset('rosters', data_path)

In [5]:
# some keys
position_key = pd.DataFrame({
    "code": [*range(1, 14), 255, 14, 15, 16, 17, 18, 19],
    "position": [
        "pitcher", "catcher", "first baseman", "second baseman", "third baseman",
        "shortstop", "left field", "center field", "right field", "batter",
        "runner on first base", "runner on second base", "runner on third base",
        "ball event with no player (e.g., ball bounce)", "home plate umpire",
        "field umpire", "field umpire", "field umpire",
        "first base coach", "third base coach"
    ]
})

event_key = pd.DataFrame({
    "code": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 16],
    "play_type": [
        "pitch", "ball acquired", "throw (ball-in-play)", "ball hit into play", 
        "end of play", "pickoff throw", "ball acquired - unknown field position", 
        "throw (ball-in-play) - unknown field position", "ball deflection", 
        "ball deflection off of wall", "home run", "ball bounce"
    ]
})

Getting all player locations for baserunners

In [11]:
player_pos_df = player_pos_ds.to_table().to_pandas()

df = player_pos_df[player_pos_df["player_position"].isin([11, 12, 13])]

df

,game_str,play_id,timestamp,player_position,field_x,field_y
4471,y1_d069_ACN_QEA,7,156603,11,56.4582,70.7073
4484,y1_d069_ACN_QEA,7,156653,11,56.4012,70.8111
4497,y1_d069_ACN_QEA,7,156703,11,56.3442,70.9146
4510,y1_d069_ACN_QEA,7,156753,11,56.2875,71.0181
4523,y1_d069_ACN_QEA,7,156803,11,56.2308,71.1216
...,...,...,...,...,...,...
76350324,y1_d058_WZR_YJD,330,12032975,12,-0.5472,-1.1253
76350325,y1_d058_WZR_YJD,330,12032975,13,14.2191,-7.3068
76350336,y1_d058_WZR_YJD,330,12033008,11,-0.2910,120.6429
76350337,y1_d058_WZR_YJD,330,12033008,12,0.1224,-1.6680


number of unique combos of game, play id, and baserunner - answer is 49,633 and next code snippet says 35,183 instances in which a player has timestamps more than 1 second apart. what type of play do we think isn't included in that?

In [12]:
# pull out just the three columns
triplets = df[["game_str", "play_id", "player_position"]]

# drop duplicate rows
unique_triplets = triplets.drop_duplicates()

# count them
num_unique = unique_triplets.shape[0]

print("Unique game/play/position combinations:", num_unique)


Unique game/play/position combinations: 49667


Finding max speed of each runner in each play between any two timestamps that are at least 1 second apart

good logic???

In [13]:
import numpy as np
import pandas as pd

# 1) sort once so each group is ordered by time
df = df.sort_values(
    ["game_str", "play_id", "player_position", "timestamp"],
    ignore_index=True
)

threshold = 1000  # milliseconds

results = []
group_cols = ["game_str", "play_id", "player_position"]

# 2) for each player‐in‐play, scan for 1+ second intervals
for (game, play, slot), gdf in df.groupby(group_cols, sort=False):
    gdf = gdf.reset_index(drop=True)
    ts = gdf["timestamp"].to_numpy()
    x  = gdf["field_x"].to_numpy()
    y  = gdf["field_y"].to_numpy()
    
    # for each i, find the earliest j with ts[j] >= ts[i] + threshold
    j_idxs = np.searchsorted(ts, ts + threshold, side="left")
    
    # keep only valid pairs
    valid = j_idxs < len(ts)
    i_idxs = np.nonzero(valid)[0]
    k_idxs = j_idxs[valid]
    
    if len(i_idxs) == 0:
        continue
    
    dt = ts[k_idxs] - ts[i_idxs]           # ms
    dx = x[k_idxs]  - x[i_idxs]
    dy = y[k_idxs]  - y[i_idxs]
    
    # speed in ft/s
    speeds = np.sqrt(dx*dx + dy*dy) / (dt / 1000)
    
    results.append({
        "game_str":          game,
        "play_id":           play,
        "player_position":   slot,
        "max_speed_ft_per_s": speeds.max()
    })

# 3) build the result DataFrame and sort descending
max_speeds = (
    pd.DataFrame(results)
      .sort_values("max_speed_ft_per_s", ascending=False)
      .reset_index(drop=True)
)

max_speeds

,game_str,play_id,player_position,max_speed_ft_per_s
0,y1_d067_IAQ_YJD,70,11,31.181173
1,y2_d071_XFE_RZQ,197,11,31.002160
2,y1_d091_DYE_RZQ,239,11,30.909686
3,y2_d043_RZQ_YJD,125,11,30.877511
4,y1_d064_UEX_RZQ,118,11,30.814254
...,...,...,...,...
35320,y2_d032_KNB_YJD,236,11,0.000000
35321,y2_d007_DYE_RZQ,104,11,0.000000
35322,y1_d011_DYE_RZQ,281,12,0.000000
35323,y1_d023_AKX_YJD,176,11,0.000000


Merged on player id and filtered out plays where speed is under 10 ft/sec - can change that bound later but ultimately want to take 90th percentile anyway so prob doesn't matter much

In [15]:
big_ie = pd.read_csv('big_ie.csv')

In [16]:
import pandas as pd

# ─── 1) Start from your big_ie DataFrame ────────────────────────────────────────
# (make sure you’ve already loaded it as `big_ie`)

# We only need one row per play to get the runner IDs:
runners = (
    big_ie
    # select the columns we care about
    [["game_str","play_id","first_baserunner","second_baserunner","third_baserunner"]]
    # drop duplicate rows so there’s one line per play
    .drop_duplicates(subset=["game_str","play_id"])
    # melt wide→long so we get one row per runner slot
    .melt(
        id_vars=["game_str","play_id"],
        value_vars=["first_baserunner","second_baserunner","third_baserunner"],
        var_name="slot_name",
        value_name="player_id"
    )
    # map slot_name to the numeric player_position
    .assign(
        player_position = lambda d: d["slot_name"].map({
            "first_baserunner":  11,
            "second_baserunner": 12,
            "third_baserunner":  13
        })
    )
    # drop any missing (e.g. innings with no runner on 3rd)
    .dropna(subset=["player_id"])
    .drop(columns="slot_name")
)

# ─── 2) Merge with your max_speeds DataFrame ────────────────────────────────────
# (which has columns game_str, play_id, player_position, max_speed_ft_per_s)
merged = (
    max_speeds
    .merge(
        runners,
        on=["game_str","play_id","player_position"],
        how="left"
    )
)

merged = merged[merged["max_speed_ft_per_s"] >= 10]

merged

,game_str,play_id,player_position,max_speed_ft_per_s,player_id
0,y1_d067_IAQ_YJD,70,11,31.181173,IAQ-1783
1,y2_d071_XFE_RZQ,197,11,31.002160,XFE-2484
2,y1_d091_DYE_RZQ,239,11,30.909686,NaN
3,y2_d043_RZQ_YJD,125,11,30.877511,RZQ-0385
4,y1_d064_UEX_RZQ,118,11,30.814254,UEX-1111
...,...,...,...,...,...
9370,y2_d044_RZQ_YJD,238,11,10.020939,YJD-0373
9371,y2_d025_ALA_YJD,200,11,10.015362,YJD-0080
9372,y2_d084_LFS_YJD,298,11,10.014888,YJD-0429
9373,y2_d006_NSO_YJD,204,11,10.012432,NaN


Why are 3000 player ids missing???

In [17]:
# how many rows have no player_id after the merge?
num_missing = merged["player_id"].isna().sum()
total_rows  = len(merged)

print(f"Missing player_id: {num_missing} rows out of {total_rows}")


Missing player_id: 3010 rows out of 9375


In [18]:
player_stats = (
    merged
    .groupby("player_id", dropna=True)
    .agg(
        num_appearances=("player_id", "size"),
        peak_speed_ft_per_s=("max_speed_ft_per_s", "max")
    )
    .reset_index()
)

player_stats


,player_id,num_appearances,peak_speed_ft_per_s
0,ACN-1139,3,21.301986
1,ACN-1147,3,25.879406
2,ACN-1180,3,25.472848
3,ACN-1276,2,22.952987
4,ACN-1338,2,19.152034
...,...,...,...
845,YUH-1903,4,24.310984
846,YUH-1918,3,24.771167
847,YUH-2350,4,29.012562
848,YUH-2406,2,25.954560


In [19]:
player_stats = (
    merged
    .groupby("player_id", dropna=True)
    .agg(
        num_appearances=("player_id", "size"),
        peak_speed_ft_per_s=("max_speed_ft_per_s", "max"),
        pct90_speed_ft_per_s=("max_speed_ft_per_s", lambda x: x.quantile(0.9))
    )
    .reset_index()
    # keep only players with at least 5 appearances
    .query("num_appearances >= 5")
    # sort by 90th percentile speed, highest first
    .sort_values("pct90_speed_ft_per_s", ascending=False)
    .reset_index(drop=True)
)

player_stats

,player_id,num_appearances,peak_speed_ft_per_s,pct90_speed_ft_per_s
0,IAQ-1783,12,31.181173,30.444060
1,IKJ-2180,9,29.627385,29.361257
2,RZQ-0385,18,30.877511,29.338936
3,XFE-2484,7,31.002160,29.190104
4,ALA-2442,5,29.474691,29.181994
...,...,...,...,...
345,YJD-0016,5,23.657485,21.925403
346,RQJ-1518,5,22.121443,21.876424
347,GHD-1074,7,23.702032,21.839530
348,NYA-2152,5,21.437443,21.313309
